<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/mbs2.png width=800 align=left>

---
## Vault Demo Data

**Prerequisites:**
- Vault binary


Before you begin, execute the following command in a separate windows:

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`


## Setup



In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault --version

In [ ]:
vault login root

In [ ]:
vault status

### Apply License

In [ ]:
sleep 5
curl \
    --insecure \
    --header "X-Vault-Token: $(vault print token)" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

## Create Policy

In [ ]:
cat << EOF > /tmp/static.hcl

path "secret1/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

path "secret/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

EOF

In [ ]:
vault policy write static-policy /tmp/static.hcl

In [ ]:
# Work with dynamic secrets engine

cat << EOF > /tmp/dynamic.hcl

path "mdb1/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

EOF

In [ ]:
vault policy write dynamic-policy /tmp/dynamic.hcl

In [ ]:
vault policy list


**Note:** in *key/value secret engine 2*, the path to write policy would be **kv/data/\<path\>** even though the CLI command to the path is **kv/\<path\>**

### Enable Secrets Engine

In [ ]:
vault secrets enable -path=secret1/ -version=1 kv

In [ ]:
vault secrets enable -path=secret2/ -version=2 kv

In [ ]:
vault kv put secret/training username="student01" password="passw0rd"

## Create Users and Associate Policies
During the configuration of users, roles, entities, and groups a list of policies can be specified

vault write auth/ldap/groups/sre policies="dev, ops"
Another example: ```vault write auth/userpass/users/tbagio password="s3cr3t" policies="base"

In [ ]:
vault auth enable userpass
vault write auth/userpass/users/arthur password="passw0rd" policies="static-policy"
vault write auth/userpass/users/samantha password="passw0rd" policies="static-policy"
vault write auth/userpass/users/tio password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/moayad password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/raymond password="passw0rd" policies="static-policy, dynamic-policy"

**List users**

In [ ]:
vault login -method=userpass username=tio password=passw0rd

In [ ]:
## how do i know leaseid other than from UI?
vault write sys/leases/lookup lease_id=auth/userpass/login/tio/h03f37d28467dcb68a4aeedea8cef88cf49103996a0fd82774dbd39a1467478a4

In [ ]:
vault list auth/userpass/users

**Delete user**

In [ ]:
vault delete auth/userpass/users/tbagio

#### Create a token attached to the newly *base* policy

In [ ]:
vault token create -policy="base"

**Note:** every token automatically gets *default* policy attached.

---
## Test "base" Policy

Using base token, you have very limited permissions.

Using base token, you have very limited permissions.  THe base policy does not have a rule on `sys/policy` path.  Lack of policy means **no permission** on that path.  Therefore, "permission denied" error is the expected behavior.

In [ ]:
VAULT_TOKEN=root vault kv put secret/training_test password="p@ssw0rd"

#### Read the data back

In [ ]:
VAULT_TOKEN=s.luiv46O7YTNZNZ3igP77aJKn vault kv get secret/training_test

#### Base policy only granst "create" and "read".  With the absence of "update" permission, this operation fails.

In [ ]:
vault kv put kv/training_test password="password1234"

In [ ]:
vault kv put kv/team-eng/apikey api_key="123455"

In [ ]:
vault kv delete kv/team-eng/apikey

In [ ]:
vault kv put kv/training_ year="2020"

&nbsp;   
### Check Token capabilities
The `vault token capabilities` command fetches the capabilities of a token for a given path.  You can review the policy, but if your token has multiple policies attached, you have to review all the associated policies which can get troublesome.

In [ ]:
vault token capabilities kv/data/apikey

In [ ]:
vault token capabilities kv/data/splunk/apikey

#### Try another path that is not permitted by the `base` policy

In [ ]:
vault token capabilities kv/data/test

In [ ]:
vault write sys/leases/lookup lease_id=mdb1/creds/engineer/nsAyHW7BzHhRVSRvt7R1G2Ak
vault token capabilities kv/data/test

---
### Policy for Policies
To allow users to be able to manage policies, you need to write policies for that

In [ ]:
cat << EOF > /tmp/manage-policies.hcl

path "sys/policies/acl/*" {
  capabilities = ["create", "read", "update", "delete", "list", "sudo"]
}
# Create and manage ACL policies via API
path "sys/policies/acl" {
  capabilities = ["list"]
}
EOF

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">